In [1]:
try:
    import requests as req
    from bs4 import BeautifulSoup 
    import re
    import pandas as pd
    import time
    from selenium import webdriver
    import numpy as np
    from requests.adapters import HTTPAdapter, Retry
    import json
    import requests
    from requests.exceptions import HTTPError
    import json
    from json.decoder import JSONArray
    import pandas as pd
    from datetime import datetime, timezone
except ImportError:
    print("No module found")
    

# Defines proxie server
proxies = {"http": "http://37.48.118.4:13081"}

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    } 
sess = req.Session()

TAXONOMY_PROD = 'http://taxonomy-management.nj.peer39.com:30080/api/'
TAXONOMY_STG = 'http://taxonomy-management.k8s.stg.peer39dom.com/taxonomy/'
TAXONOMY_LOCAL = 'http://localhost:8080/'
TAXONOMY_URL = TAXONOMY_PROD


config = open("Utils_Roku_For_Crawler.json")
config_file = json.load(config)
print (json.dumps(config_file, indent=4))

{
    "char_to_replace": {
        "|": " ",
        "-": " ",
        "\u2013": " ",
        "(": " ",
        ")": " ",
        ",": " ",
        "!": " ",
        "_": " ",
        ".": " ",
        ":": " ",
        "\\": " ",
        "\n": " ",
        "\t": " ",
        "\u00e9": "e",
        "\u00e1": "a",
        "\u00e4": "a",
        "\u00f6": "oo",
        "\u00fc": "u",
        "\u00e0": "a",
        "\u00e8": "e",
        "\u00ec": "i",
        "\u00f2": "o",
        "\u00f9": "u",
        "\u00e7": "c",
        "\u00ed": "i",
        "\u00f3": "o",
        "\u00fa": "u",
        "\u00e3": "a",
        "\u00ea": "e",
        "\u00f4": "o",
        "\u00e2": "a",
        "\u00f5": "o"
    },
    "list_of_NonOtt_categories": [
        "Games",
        "Music & Podcasts",
        "Apps"
    ],
    "categories_word_to_category": {
        "news": "news",
        "movies": "Movies",
        "TV shows": "TV Shows",
        "movie": "Movies",
        "TV show": "TV Shows",
      

In [24]:
driver = webdriver.Chrome('\\Users\Eran.Braude\Downloads\chromedriver_win32/chromedriver.exe')

In [32]:
# ID_LIST = ["f57e65ec03026501147bd737caabed83"] #["614780", "96da35e0bce6c184b61e445cc6e62203"]
sample = pd.read_excel('C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Roku_Sample_Check_200.xlsx', squeeze = True)
ID_LIST = sample['dsp_app_id'].to_list()

In [ ]:
ID_LIST

In [2]:
def loginReq(TAXONOMY_URL):
    # post req - login
    login = requests.post(TAXONOMY_URL + 'login', json={
#         "username": "QaAdminUser",
#         "password": "Password1"
        # "username": "TestRan343",
        # "password": "Sonicmario3$"
        "username": "webology_user",
        "password": "Webology20"
    })
    return login

# in case you want to check only the login, open the following and run code until here.
login = loginReq(TAXONOMY_URL)
print (login)  

<Response [200]>


In [3]:
def bring_long_id(param_id):
    '''
    desc: The function will return hashed Roku store id, based on a data frame that mapping between short id and long id.
    param: id as it comes from ID_LIST.
    return: long id.
    '''
    df = pd.read_csv("C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Roku_All_id.csv", squeeze = True)
    df['Roku_id'] = df['Roku_id'].astype(str)
    if(len(param_id) > 10):
        return param_id
    else:
        try:
            return df[df['Roku_id'] == f'{param_id}.0']['long_id'].tolist()[0]
        except Exception as e:
            print(f"{e}")
            return -1 

In [4]:
bring_long_id('63402')

'8b5b60b5e910802e55af2274311b77af'

In [5]:
def send_API_build_app_obj(param_id):
    '''
    desc: The function will create an URL and send it to the Roku API. Process the response and make the initial app object.
    param: long id.
    return: initial app object.
    '''
    categories_list = []
    
    url_to_api = f'https://channelstore.roku.com/api/v6/channels/{param_id}'
    driver.get(f'https://channelstore.roku.com/en-gb/details/{param_id}/cbc-tv')
    time.sleep(1)
    driver.get(url_to_api)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    app_json = soup.find('pre').text
    response = json.loads(app_json)
    
#     response = json.loads(sess.get(url_to_api, headers = headers, proxies = proxies).content)
#     print (json.dumps(response, indent=4))

    categories = response.get('categoryList', [])
    if(categories != None):
        for category in categories:
            categories_list.append(category['name'])
            
    app_object = {'appstore_app_id' : response.get('id', ''),
                 'appstore_app_name' : response.get('name', ''),
                 'app_long_id' : param_id,
                 'app_average_rating' : (response['rating'] / 10) / 2, #"{:.1f}".format(42.25478654)
                 'app_num_raters' : response.get('starRatingCount', ''),
                 'app_description' : response.get('webDescription', ''),
                 'ctv_store_name' : 'Roku',
                 'ctv_store_id' : 3,
                 'ott_device_platform_ctv' : 'Roku',
                'ott_content_categories' : categories_list,
                 'appstore_link' : f'https://channelstore.roku.com/details/{param_id}',
                 'developer_name' : response.get('developerName', ''),
                 'developer_page' : response['currentDetail']['developerPrivacyUrl'],
                 }
    
    print (json.dumps(app_object, indent=4))
    return app_object

In [5]:
app_obj = send_API_build_app_obj('9ce92aaafddcaa4f5e1422ffc7cc0436')

NameError: name 'driver' is not defined

In [6]:
def clean_text_from_special_chars(text):
    '''
    desc: The function clean text from secial chars and NON alpha numeric values.
    param: some unclean text.
    return: clean text.
    '''
    char_to_replace = config_file['char_to_replace']
    
#     {'|' : ' ',
#                        '-': ' ',
#                        '–': ' ',
#                        '(': ' ',
#                        ')': ' ',
#                     ',': ' ',
#                    '!': ' ',
#                    '-': ' ',
#                     '_' : ' ',
#                     '.' : ' ',
#                       ':' : ' ',
#                        'é' : 'e',
#                         'á' : 'a',
#                   'ä' : 'a',
#                   'ö':'oo',
#                   'ü':'u',
#                   'à':'a',
#                   'è':'e',
#                   'ì':'i','ò':'o','ù':'u','é':'e','ç':'c','á':'a','í':'i','ó':'o','ú':'u','ã':'a','ê':'e','ô':'o','â':'a','õ':'o', 'ó':'o' }
    
    text_clean = text.translate(str.maketrans(char_to_replace))
    text_clean = re.sub("[^A-Za-z0-9\s\+\&]","",text_clean)
    text_clean = " " + text_clean.lower() + " "
    text_clean = re.sub(' +', ' ', text_clean) 
    
    return text_clean

In [7]:
clean_text_from_special_chars("hi? is netflix! Eran!!!ddd'")

' hi is netflix eran ddd '

In [8]:
def app_store_name_check_if_exist_in_our_db(app_obj):
    '''
    desc: The function checks if the app_name contains a channel / OTT that exists in our DB. If true, it will return the enriched object.
    param: app_obj that was created in send_API_build_app_obj function.
    return: the app_object (with / without new data) + True if we did find it in our DB or False if we didnt.
    '''
    
    data_base = pd.read_excel('C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Appstore_name_classifire\\full_data.xlsx', squeeze = True)
    data_base = data_base.fillna(0)
    data_base['phrase_id'] = data_base['phrase_id'].astype(int)
    data_base['alias'] = [i.strip() for i in data_base['alias'].tolist()]
    
    app_obj['appstore_app_name_clean'] = clean_text_from_special_chars(app_obj['appstore_app_name'])    
        
    for word in data_base['alias'].tolist():
        if f' {word.lower()} ' == app_obj['appstore_app_name_clean']:
            app_obj['id_in_db'] = data_base[data_base['alias'] == word]['id-2'].item()
            break
        elif f' {word.lower()} ' in app_obj['appstore_app_name_clean']:
            app_obj['id_in_db'] = data_base[data_base['alias'] == word]['id-2'].item()
            break
        else:
            app_obj['id_in_db'] = -1
            
    if(app_obj['id_in_db'] != -1):
        app_obj['ott_channel_content_type'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['phrase'].item() if data_base[data_base['id-2'] == app_obj['id_in_db']]['type'].item() == 'Channel' else None
        app_obj['ott_service'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['phrase'].item() if data_base[data_base['id-2'] == app_obj['id_in_db']]['type'].item() == 'OTT_Service' else None
        app_obj['ottProductionType'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['ott_production_type'].item()
        app_obj['ottBroadcastType'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['ott_broadcast_type'].item()
        app_obj['ottRevenueModel'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['ottRevenueModel'].item()
        app_obj['ott_content_categories'] = data_base[data_base['id-2'] == app_obj['id_in_db']]['ott_content_categories'].item()
        return app_obj, True
    else:
        return app_obj, False

In [9]:
def ott_or_not_ott_by_app_categories(app_obj):
    '''
    desc: The function checks if te app is OTT or not OTT based on the APP categories.
    param: app_obj that was created in send_API_build_app_obj function.
    return: True if the Object is NON_OTT or False if otherwise.
    '''
    
    list_of_NonOtt_categories = config_file['list_of_NonOtt_categories']
#     ['Games', 'Music & Podcasts', 'Apps']
    ind = 0
    
    for category in app_obj['ott_content_categories']:
        for cstegory_in_non_ott_list in list_of_NonOtt_categories:
            if(category.lower() == cstegory_in_non_ott_list.lower()):
                ind = ind + 1
        else:
            ind = ind + 0
            
    return True if ind > 0 else False

In [10]:
def bring_Non_OTT_conent_categories(app_obj):
    
    content_categories = []
    
    categories_word_to_category_non_ott = config_file['categories_word_to_category_non_ott']
    
        
    app_obj['app_description_clean'] =  clean_text_from_special_chars(app_obj['app_description'])       
 
    
    for category in app_obj["ott_content_categories"]:
        if(category == 'Games'):
            content_categories.append('Games')
            content_categories.append('CTV_nonOTT')
            app_obj["ott_content_categories"] = content_categories
            app_obj['ctv_store_name'] = 'Roku_nonOTT'
            app_obj['ctv_store_id'] = ''
            app_obj['ott_device_platform_ctv'] = None
            return app_obj
            
        if(category == 'Music & Podcasts'):
            content_categories.append('Radio and Podcasts')
            content_categories.append('CTV_nonOTT')
            app_obj["ott_content_categories"] = content_categories
            app_obj['ctv_store_name'] = 'Roku_nonOTT'
            app_obj['ctv_store_id'] = ''
            app_obj['ott_device_platform_ctv'] = None
            return app_obj
        
    for word_to_check in categories_word_to_category_non_ott:
        if f' { word_to_check.lower() } ' in app_obj['app_description_clean'].lower():
            content_categories.append(categories_word_to_category_non_ott[word_to_check])
            content_categories.append('CTV_nonOTT')
            content_categories = list(dict.fromkeys(content_categories))
            app_obj["ott_content_categories"] = content_categories
    
    if(len(content_categories) == 0):
        content_categories.append('Other')
        content_categories.append('CTV_nonOTT')
        content_categories = list(dict.fromkeys(content_categories))
        app_obj["ott_content_categories"] = content_categories
    
    app_obj['ctv_store_name'] = 'Roku_nonOTT'
    app_obj['ctv_store_id'] = ''
    app_obj['ott_device_platform_ctv'] = None

    return app_obj

In [11]:
def ott_or_not_ott_by_app_name_desc(app_obj):
    '''
    desc: The function checks if te app is OTT or not OTT based on the APP name and app description.
    param: app_obj that was created in send_API_build_app_obj function.
    return: True if the Object is NON_OTT or False if otherwise.
    '''
    ind = 0

    
    NON_OTT_KWS_LIST_FIRST_LALYER = config_file['NON_OTT_KWS_LIST_FIRST_LALYER']
    
#     ['card', 'cards', 'card game', 'game', 'games', 'Podcasts', 'Podcast', 'puzzles',
#                         'Solitaire', 'gamer', 'screensaver', 'Wallpapers', 'Wallpaper', 'screensavers', 'Cast', 'casting', 'video casting',
#                        'Screen Mirroring', 'Share photos', 'casting', 'Caster']
    
    OTT_KWS_LIST = config_file['OTT_KWS_LIST']
    
#     ['TV', 'stream', 'streaming', 'streamers', 'broadcast', 'Shows', 'Show', 'episodes', 'episode', 'channel']
    
    NON_OTT_KWS_SECOND_LALYER = config_file['NON_OTT_KWS_SECOND_LALYER']
    
#     ["tunein", "Radio", "Internet Radio", "Radios", 'CAST TV', 'TV Remote', 'casting', 'Podcasts', 'Podcast', 'cast']
    
    OTT_KWS_LIST_SECOND_LAYER = config_file['OTT_KWS_LIST_SECOND_LAYER']
    
#     ['TV Network']
    
    app_obj['appstore_app_name_clean'] = clean_text_from_special_chars(app_obj['appstore_app_name']) 
    app_obj['app_description_clean'] =  clean_text_from_special_chars(app_obj['app_description'])
    
    for word in NON_OTT_KWS_LIST_FIRST_LALYER:
        if f' {word.lower()} ' in app_obj['appstore_app_name_clean'].lower() or f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind = 1
            
    for word in OTT_KWS_LIST:
        if f' {word.lower()} ' in app_obj['appstore_app_name_clean'].lower() or f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind = 0
            
    for word in NON_OTT_KWS_SECOND_LALYER:
        if f' {word.lower()} ' in app_obj['appstore_app_name_clean'].lower() or f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind = 1
    
    for word in OTT_KWS_LIST_SECOND_LAYER:
        if f' {word.lower()} ' in app_obj['appstore_app_name_clean'].lower() or f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind = 0
        
    print(json.dumps(app_obj, indent=4))
    print(ind)
    
    if(ind == 0):
        return False
    else:
        return True

In [12]:
def bring_app_properties(app_obj):
    '''
    desc: The function will activate sub functions to bring the app properties.
    param: app_obj that was created in send_API_build_app_obj function.
    return: enriched app with all the properties.
    '''
    
    app_obj['app_description_clean'] = clean_text_from_special_chars(app_obj['app_description'])
    app_obj = bring_app_properties_categories(app_obj)
    app_obj = bring_app_properties_revenue_model(app_obj)
    app_obj = bring_app_properties_broadcast_type(app_obj)
    app_obj = bring_app_properties_production_type(app_obj)
    
    print(json.dumps(app_obj, indent=4))
    
    return app_obj

In [13]:
def bring_app_properties_categories(app_obj):
    
    categories_word_to_category = config_file['categories_word_to_category']
    
#     {'news' : 'news', 'movies' : 'Movies', 'TV shows' : 'TV Shows', 'movie' : 'Movies', 'TV show' : 'TV Shows',
#                                   'documentaries' : 'Documentary', 'Documentary' : 'Documentary', 'kids' : 'Kids', 'Entertainment' : 'Entertainment',
#                                   'anime' : 'Anime', 'shows' : 'TV Shows', 'family' : 'Family', 'game shows' : 'Game Shows', 'news' : 'news', 'sports' : 'Sports',
#                                   'music' : 'Music', 'educational' : 'Education', 'cartoons' : 'Cartoon', 'cinema' : 'Movies',
#                                   'Film' : 'Movies', 'all genres' : 'Entertainment', 'WWE' : 'Boxing', 'WCW' : 'Boxing', 'ECW' : 'Boxing',
#                                   'ring shows' : 'Sports', 'food' : 'Food&Beverage', 'food shows' : 'Food&Beverage', 'cooking' : 'Food&Beverage', 'recipes' : 'Food&Beverage', 'cuisines' : 'Food&Beverage', 'vegan' : 'Food&Beverage', 'foodies' : 'Food&Beverage',
#                                   'cartoon' : 'Cartoon', 'parent friendly' : 'Kids', 'Toon' : 'Cartoon', 'kidSAFE' : 'Kids', 'Headlines' : 'news',
#                                   'Journalist' : 'news', 'breaking news' : 'news', 'weather' : 'Weather', 'celebrities' : 'Celebrities and Gossip', 'reality programming' : 'Reality', 'home improvement' : 'Home&Garden', 'Horror' : 'Horror', 'Sci Fi' : 'Sci-Fi',
#                                   'films' : 'Movies', 'slashers' : 'Horror', 'docuseries' : 'Documentary', "Christian" : 'Religion', 'Faith' : 'Religion', 'Fitness' : 'Fitness', 'workouts' : 'Fitness', 'Church' : 'Religion', 'NFL' : 'AmericanFootball', 'comedies' : 'Comedy', 'Comedy' : 'Comedy',
#                                   'reality TV' : 'Reality', 'reality shows' : 'Reality', 'Fishing' : 'Recreation', 'series' : 'TV Shows', 'Gospel' : 'Religion',
#                                   'Catholic' : 'Religion', 'Entertaining' : 'Entertainment'}
    
    categories_list = []
    
    for word in categories_word_to_category:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            categories_list.append(categories_word_to_category[word])
    
    if(len(categories_list) > 0):
        categories_list = list(dict.fromkeys(categories_list))
        app_obj["ott_content_categories"] = categories_list
        
    else:
        categories_list_sec = []
        categories_list = []
        for roku_category in app_obj["ott_content_categories"]:
            for classifier_category in config_file['content_categories_to_category']:
                if(roku_category == classifier_category):
                    categories_list.append(config_file['content_categories_to_category'][classifier_category])
                    
        for cat_list in categories_list:
            for categoey in cat_list:
                categories_list_sec.append(categoey)
        
        app_obj["ott_content_categories"] = categories_list_sec

        
    return app_obj

In [15]:
def bring_app_properties_revenue_model(app_obj):
    
    ind_SVOD = 0
    ind_AVOD = 0

    
    categories_word_to_revenue_model_SVOD = config_file['categories_word_to_revenue_model_SVOD']
    
#     ['to subscribe', 'No commercials', 'subscription plan', 'subscribe', 'Subscriber Agreement', 'Subscriber', "buy", "rent", "premium ad free", 'without the ads', 'Monthly Subscription', 'Free Trial', 'Payment is charged', 'subscription automatically renews', 'access credentials']
    categories_word_to_revenue_model_AVOD = config_file['categories_word_to_revenue_model_AVOD']
#     ['unlimited FREE', 'absolutely free', 'no subscription', 'free movies', 'free streaming service', 'without a subscription', 'FREE movies and TV', 'no Payment', 'free from Payment', 'free of Payment', 'unlimited FREE', 'No credit card']
    
    for word in categories_word_to_revenue_model_SVOD:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind_SVOD = ind_SVOD + 1
    
    for word in categories_word_to_revenue_model_AVOD:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind_AVOD = ind_AVOD + 1
            
    
    if(ind_SVOD > 0 and ind_AVOD > 0): 
        app_obj['ottRevenueModel'] = "Mixed" 
    elif(ind_SVOD > 0):
        app_obj['ottRevenueModel'] = "SVOD" 
    elif(ind_AVOD > 0):
        app_obj['ottRevenueModel'] = "AVOD"
    else:
        app_obj['ottRevenueModel'] = "" 
    
    return app_obj

In [16]:
def bring_app_properties_broadcast_type(app_obj):
    
    ind_LIVE = 0
    ind_VOD = 0

    
    categories_word_to_revenue_model_LIVE = config_file['categories_word_to_revenue_model_LIVE']
    
#     ['24 hour news', 'live', 'Live TV Channels', 'Live TV', 'watch LIVE']
    
    categories_word_to_revenue_model_VOD = config_file['categories_word_to_revenue_model_VOD']
    
#     ['video catch up', 'on demand', 'collection of movies', 'catalog', 'video content', 'video', 'download', 'video on demand', 'binge', 'catch up']
    
    for word in categories_word_to_revenue_model_LIVE:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind_LIVE = ind_LIVE + 1
    
    for word in categories_word_to_revenue_model_VOD:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            ind_VOD = ind_VOD + 1
            
    
    if(ind_LIVE > 0 and ind_VOD > 0): 
        app_obj['ottBroadcastType'] = "Mixed" 
    elif(ind_VOD > 0):
        app_obj['ottBroadcastType'] = "VOD" 
    elif(ind_LIVE > 0):
        app_obj['ottBroadcastType'] = "LIVE"
    else:
        app_obj['ottBroadcastType'] = "VOD" 
    
    return app_obj

In [17]:
def bring_app_properties_production_type(app_obj):
    
    words_of_UGC_content = config_file['words_of_UGC_content']
    words_of_PRO_content = config_file['words_of_PRO_content']
    
#     ['movies', 'Live TV Channels', 'Live Channels', 'TV shows', 'classics', 'anime', 'documentaries',
#                            'Live TV', 'award winning', 'series', 'business news', 'entertainment network', 'global news',
#                            'live video stream', 'breaking news', 'News app', 'Full episodes', 'news service', 'news broadcasting',
#                            'video catch up', 'Popular Shows']
    
    # ind o = UGC, ind 1 = Professional 
    num_of_raters = app_obj.get('app_num_raters', 0)
    desc_len = len(app_obj.get('app_description', ''))
    ind_words_of_UGC_content = 0
    ind_words_of_PRO_content = 0
        
    for word in words_of_UGC_content:
        if f' {word.lower()} ' in app_obj.get('app_description_clean', '').lower():
            ind_words_of_UGC_content = ind_words_of_UGC_content + 1
    
    for word in words_of_PRO_content:
        if f' {word.lower()} ' in app_obj.get('app_description_clean', '').lower():
            ind_words_of_PRO_content = ind_words_of_PRO_content + 1
    
    if(num_of_raters <= 10 and desc_len <= 15 and ind_words_of_PRO_content == 0):
        app_obj['ottProductionType'] = 'UGC'
    elif(ind_words_of_UGC_content > 0 and ind_words_of_PRO_content > 0):
        app_obj['ottProductionType'] = 'PRO-AM'
    elif(ind_words_of_PRO_content > 0):
        app_obj['ottProductionType'] = 'Professional'
    elif(ind_words_of_UGC_content > 0):
        app_obj['ottProductionType'] = 'UGC'
    else:
        app_obj['ottProductionType'] = 'Professional'
    
    return app_obj

In [18]:
def preper_object_to_api(app_obj):
    
    category_str = ''
    
    try:
        del app_obj["app_description_clean"]
    except Exception as e:
        None
    
    try:
        del app_obj["appstore_app_name_clean"]
    except Exception as e:
        None
    
    try:
        del app_obj["id_in_db"]
    except Exception as e:
        None
        
    app_obj['dspAppId'] = str(app_obj.pop('appstore_app_id', None))
    app_obj['appstoreAppName'] = app_obj.pop('appstore_app_name', None)
    app_obj['appstoreAppId'] = app_obj.pop('app_long_id', None)
    app_obj['channelLink'] = app_obj.pop('appstore_link', None)
    app_obj['insertDate'] = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
    app_obj['updateDate'] = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")
    app_obj['type'] = 'Manual'
    app_obj['appAverageRating'] = app_obj.pop('app_average_rating', None)
    app_obj['appNumRaters'] = app_obj.pop('app_num_raters', None)
    app_obj['appDescription'] = app_obj.pop('app_description', None)
    app_obj['ctvStoreName'] = app_obj.pop('ctv_store_name', None)
    app_obj['ctvStoreId'] = app_obj.pop('ctv_store_id', None)
    app_obj['ottDevicePlatformCtv'] = app_obj.pop('ott_device_platform_ctv', None)   
    app_obj['ottChannelContentType'] = app_obj.pop('ott_channel_content_type', None)
    app_obj['ottService'] = app_obj.pop('ott_service', None)

    
    if isinstance(app_obj["ott_content_categories"], list):
        for category in app_obj["ott_content_categories"]:
            category_str = category_str + category + ";"
        app_obj["ott_content_categories"] = category_str
        
    app_obj['ottContentCategories'] = app_obj.pop('ott_content_categories', None)
    
    print(json.dumps(app_obj, indent=4))
    
    return app_obj

# [
#     {
#         "dspAppId": "E184833LALAKJ",
#         "appstoreAppName": "Church Services TV",
#         "appstoreAppId": "b1849971f841d4c61f35a14fbf6f8b91",
#         "insertDate": "2023-03-05T15:34:48.171175Z",
#         "updateDate": "2023-03-05T15:34:48.171175Z",
#         "type":"Manual",
#         "appAverageRating": 4.075,
#         "appNumRaters": 40,
#         "appDescription": "Church Services TV have been streaming from Churches since 2007. Every day and every hour you can watch LIVE, Churches from all over the UK, Ireland, and the USA. ",
#         "ctvStoreName": "Roku",
#         "ctvStoreId": 3,
#         "ottDevicePlatformCtv": "Roku",
#         "ottContentCategories": "Religion;",
#         "appstore_link": "https://channelstore.roku.com/details/b1849971f841d4c61f35a14fbf6f8b91",
#         "developer_name": "Oli Tormey",
#         "developer_page": "null",
#         "ottRevenueModel": null,
#         "ottBroadcastType": "LIVE",
#         "ottProductionType": "UGC",
#         "ottChannelContentType": null,
#         "ottService" : null

#     }
# ]

# {'appstore_app_id': 184833,
#  'appstore_app_name': 'Church Services TV',
#  'app_long_id': 'b1849971f841d4c61f35a14fbf6f8b91',
#  'app_average_rating': 4.075,
#  'app_num_raters': 40,
#  'app_description': 'Church Services TV have been streaming from Churches since 2007. Every day and every hour you can watch LIVE, Churches from all over the UK, Ireland, and the USA. ',
#  'ctv_store_name': 'Roku',
#  'ctv_store_id': 3,
#  'ott_device_platform_ctv': 'Roku',
#  'ott_content_categories': ['Religion', 'Movies', 'TV SHOWS'],
#  'appstore_link': 'https://channelstore.roku.com/details/b1849971f841d4c61f35a14fbf6f8b91',
#  'developer_name': 'Oli Tormey',
#  'developer_page': None,
#  'appstore_app_name_clean': ' church services tv ',
#  'id_in_db': -1,
#  'app_description_clean': ' church services tv have been streaming from churches since 2007 every day and every hour you can watch live churches from all over the uk ireland and the usa ',
#  'ottRevenueModel': '',
#  'ottBroadcastType': 'LIVE',
#  'ottProductionType': 'UGC'

In [19]:
def insertToApp(app_obj_list):
    
    json_str = json.dumps(app_obj_list, indent=4) 
    
    print(json_str)
    
    json_create_list = json.loads(json_str)
    
    login = loginReq(TAXONOMY_URL)
    
    try:
        # post create account category request
        res = requests.post('http://taxonomy-management.nj.peer39.com:30080/api/taxonomy' + '/ctv/app',
                            json=json_create_list, headers={'Content-Type': 'application/json'},
                            cookies=login.cookies)

        if res.ok:
            print("new app was created successfully", res.json())
        else:
            print("Error creating new app:\n", res.json())
    except json.decoder.JSONDecodeError as err:
        print("Error decoding JSON!", err)    

In [ ]:
app_obj = send_API_build_app_obj('a5a48d78669363f2e2ca893a0a782ce0')

In [19]:
x = bring_long_id('a5a48d78669363f2e2ca893a0a782ce0')

In [ ]:
obj, status = app_store_name_check_if_exist_in_our_db({
    "appstore_app_id": 586546,
    "appstore_app_name": "hi? is ....?CBS! Eran",
    "app_long_id": "136db0bee0360139f81fba8e7e95786d",
    "app_average_rating": 3.371795,
    "app_num_raters": 78,
    "app_description": "A gamer favorite for years!  Place all the cards in each suit in stacks of ascending order.",
    "ctv_store_name": "Roku",
    "ctv_store_id": 3,
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [
        "Games"
    ],
    "appstore_link": "https://channelstore.roku.com/details/136db0bee0360139f81fba8e7e95786d",
    "developer_name": "PlayWorks Digital LTD ",
    "developer_page": "https://www.play.works/privacy-policy"
})

print(json.dumps(obj, indent=4))
print(status)

In [ ]:
ott_or_not_ott_by_app_categories({
    "appstore_app_id": 586546,
    "appstore_app_name": "hi? is xxx! Eran",
    "app_long_id": "136db0bee0360139f81fba8e7e95786d",
    "app_average_rating": 3.371795,
    "app_num_raters": 78,
    "app_description": "PodcastsA gamer favorite for years!  Place all the cards in each suit in stacks of ascending order.",
    "ctv_store_name": "Roku",
    "ctv_store_id": 3,
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [
        "Test",
        "Games"
    ],
    "appstore_link": "https://channelstore.roku.com/details/136db0bee0360139f81fba8e7e95786d",
    "developer_name": "PlayWorks Digital LTD ",
    "developer_page": "https://www.play.works/privacy-policy"
})

In [ ]:
obj = bring_Non_OTT_conent_categories({
    "appstore_app_id": 586546,
    "appstore_app_name": "Speed Extreme",
    "app_long_id": "136db0bee0360139f81fba8e7e95786d",
    "app_average_rating": 3.371795,
    "app_num_raters": 78,
    "app_description": "In Speed Extreme, jump behind the wheel and race through the endless busy roads and highways! Get ready for the excitement and thrill of highway car racing where you can outrun cars and trucks at high speed! Speed Extreme is a classic arcade racing game where the main goal is to overtake other vehicles without crashing your own car! Be careful! Roads and highways are full of speeding vehicles, dodge them carefully by taking proper turns! Become the master of highway racing and car driving! Avoid crashes, win the race, and become the ultimate racer of the city! Features: - Amazing tracks. - Fast and addictive gameplay. - Different sports cars to choose from! - Smooth controls. - Amazing music and sound effects. - Classic arcade racing! Download Speed Extreme now and enjoy this classic arcade racing game! ",
    "ctv_store_name": "Roku",
    "ctv_store_id": 3,
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [
        "Test",
        "Movies",
        "podcasts",
        "Gamess"
        
    ],
    "appstore_link": "https://channelstore.roku.com/details/136db0bee0360139f81fba8e7e95786d",
    "developer_name": "PlayWorks Digital LTD ",
    "developer_page": "https://www.play.works/privacy-policy"
})

print (json.dumps(obj, indent=4))

In [ ]:
ott_or_not_ott_by_app_name_desc({
    "appstore_app_id": 586546,
    "appstore_app_name": "Subgenres",
    "app_long_id": "136db0bee0360139f81fba8e7e95786d",
    "app_average_rating": 3.371795,
    "app_num_raters": 78,
    "app_description": "Test !?.+ wallpaper",
    "ctv_store_name": "Roku_nonOTT",
    "ctv_store_id": "",
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [
        "Screensavers Wallpapers and Themes",
        "CTV_nonOTT",
        "Casting Beaming and Media Players"
    ],
    "appstore_link": "https://channelstore.roku.com/details/136db0bee0360139f81fba8e7e95786d",
    "developer_name": "PlayWorks Digital LTD ",
    "developer_page": "https://www.play.works/privacy-policy",
    "app_description_clean": " Steampunks! We are here for you! Subgenres is a channel that celebrates the amazing multiverse of Steampunk and the vast array of subgenres that exist within it. This channel offers programming focused on costuming, DIY upcycled props, historic and reimagined pasts/futures, creatives, music, performers, news, events, festivals, conventions, food, and a vast array of related content. Subgenres channel is for all Steampunks, we Steampunk around the world! Here's looking at you, you beautiful Steamirific world! Join us daily for your dose of steam, we endeavour to rock your clocks and grinds your cerebral gears with awesome content from all over the globe. " 
})

In [ ]:
bring_app_properties({
    "appstore_app_id": 586546,
    "appstore_app_name": "Plex - Free Movies & TV",
    "app_long_id": "136db0bee0360139f81fba8e7e95786d",
    "app_average_rating": 0,
    "app_num_raters": 55,
    "app_description": "svod movies designDownload the app to access WWE Network and stream EVERY LIVE PAY-PER-VIEW and thousands of hours of video on demand including: • Every WWE, WCW and ECW pay-per-view in history • Exclusive weekly in-ring shows • Original series and documentaries • Every episode of Raw, SmackDown and NXT In addition, get behind-the-scenes access to your favorite WWE Superstars and Legends. oku_nonOTT",
    "ctv_store_id": "",
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [
        "Movies",
        "Arts & Culture"
    ],
    "appstore_link": "https://channelstore.roku.com/details/136db0bee0360139f81fba8e7e95786d",
    "developer_name": "PlayWorks Digital LTD ",
    "developer_page": "https://www.play.works/privacy-policy",
    "appstore_app_name_clean": " UGC ",
    "app_description_clean": " movies design ",
    "ottRevenueModel": "",
    "ottBroadcastType": "",
    "ottProductionType": "Professional"
})

In [ ]:
preper_object_to_api({'appstore_app_id': 184833,
 'appstore_app_name': 'Church Services TV',
 'app_long_id': 'b1849971f841d4c61f35a14fbf6f8b91',
 'app_average_rating': 4.075,
 'app_num_raters': 40,
 'app_description': 'Church Services TV have been streaming from Churches since 2007. Every day and every hour you can watch LIVE, Churches from all over the UK, Ireland, and the USA. ',
 'ctv_store_name': 'Roku',
 'ctv_store_id': 3,
 'ott_device_platform_ctv': 'Roku',
 'ott_content_categories': ['Religion', 'Movies', 'TV SHOWS'],
 'appstore_link': 'https://channelstore.roku.com/details/b1849971f841d4c61f35a14fbf6f8b91',
 'developer_name': 'Oli Tormey',
 'developer_page': None,
 'appstore_app_name_clean': ' church services tv ',
 'id_in_db': -1,
 'app_description_clean': ' church services tv have been streaming from churches since 2007 every day and every hour you can watch live churches from all over the uk ireland and the usa ',
 'ottRevenueModel': '',
 'ottBroadcastType': 'LIVE',
 'ottProductionType': 'UGC'})

In [56]:
ID_LIST = ['61250']

In [57]:
all_apps_list = []
all_apps_list_ready_for_api = []
counter = 0
driver = webdriver.Chrome('\\Users\Eran.Braude\Downloads\chromedriver_win32/chromedriver.exe')

for one_id in ID_LIST:
    
    print(f'Iteration #{counter} out of #{len(ID_LIST)}')
    counter += 1
    
    roku_id = bring_long_id(one_id)
    
    if(roku_id == -1):
        continue
    
    try:
        app_object = send_API_build_app_obj(roku_id)
    except Exception as e:
        continue
    
    app_object, status = app_store_name_check_if_exist_in_our_db(app_object)
    
    if(status == True):
        all_apps_list.append(app_object)
        continue 
    
    status = ott_or_not_ott_by_app_categories(app_object)
    
    if(status == True):
        app_object = bring_Non_OTT_conent_categories(app_object)
        all_apps_list.append(app_object)
        continue
    
    status = ott_or_not_ott_by_app_name_desc(app_object)
    
    if(status == True):
        app_object = bring_Non_OTT_conent_categories(app_object)
        all_apps_list.append(app_object)
        continue
    
    app_object = bring_app_properties(app_object)
    
    all_apps_list.append(app_object)
    
for app in all_apps_list:
    all_apps_list_ready_for_api.append(preper_object_to_api(app))

df = pd.DataFrame.from_records(all_apps_list_ready_for_api)

# print("Done")

FILE_PATH = f'C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Roku_FIN_OUTPUT_{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}.csv'

df.to_csv(FILE_PATH, encoding='utf-8', index=False)

time.sleep(3)
    
insertToApp(all_apps_list_ready_for_api)

driver.close()
driver.quit()

Iteration #0 out of #1
{
    "appstore_app_id": 61250,
    "appstore_app_name": "First Coast News Jacksonville",
    "app_long_id": "1f5f12c161a92f7e37f8cdfbdfbb7ceb",
    "app_average_rating": 0.0,
    "app_num_raters": 182,
    "app_description": "Stay up-to-date with the latest news and weather in the Jacksonville, FL, area on the all-new First Coast News+ app from WTLV. Our channel features the latest breaking news impacting you and your family, plus video coverage from local events. First Coast News+ has the latest in sports from your favorite local pro and college teams, plus entertainment content if you want to kick back and relax. Know what's real or fake with our award-winning news verification program VERIFY. \u200bDownload NOW.\n",
    "ctv_store_name": "Roku",
    "ctv_store_id": 3,
    "ott_device_platform_ctv": "Roku",
    "ott_content_categories": [],
    "appstore_link": "https://channelstore.roku.com/details/1f5f12c161a92f7e37f8cdfbdfbb7ceb",
    "developer_name": "Teg

In [45]:
df

""


In [48]:
all_apps_list_ready_for_api

[{'developer_name': 'VIDGO',
  'developer_page': 'https://www.vidgo.com/privacy',
  'ottProductionType': 'Professional',
  'ottBroadcastType': 'Mixed',
  'ottRevenueModel': 'svod',
  'dspAppId': '277866',
  'appstoreAppName': 'Vidgo',
  'appstoreAppId': '1611455d21bf04894876bc09707a194c',
  'channelLink': 'https://channelstore.roku.com/details/1611455d21bf04894876bc09707a194c',
  'insertDate': '2023-04-16T07:22:53.379999Z',
  'updateDate': '2023-04-16T07:22:53.379999Z',
  'type': 'Manual',
  'appAverageRating': 0.0,
  'appNumRaters': 0,
  'appDescription': 'Score the best deal on live sports and TV! Enjoy in HD on your big screen or on the go! Watch all the ESPN & FOX channels at the lowest price available. Access to 80+ national & local channels featuring your favorite sports and TV shows. You can also choose from our different Latino packages. \n\nPair with the mobile Vidgo app where you can chat and interact with others while watching TV. Sign-up on https://www.vidgo.com.\n\nVidgo p

In [39]:
insertToApp(all_apps_list_ready_for_api)

[
    {
        "developer_name": "Freeform",
        "developer_page": "https://disneyprivacycenter.com",
        "ottProductionType": "Professional",
        "ottBroadcastType": "Mixed",
        "ottRevenueModel": 0,
        "dspAppId": "73357",
        "appstoreAppName": "Freeform - Movies & TV Shows",
        "appstoreAppId": "80de531382d6a805c1336e42814aa464",
        "channelLink": "https://channelstore.roku.com/details/80de531382d6a805c1336e42814aa464",
        "insertDate": "2023-04-16T07:18:12.621379Z",
        "updateDate": "2023-04-16T07:18:12.621379Z",
        "type": "Manual",
        "appAverageRating": 0.0,
        "appNumRaters": 0,
        "appDescription": "STREAM MOVIES: Watch a variety of movies from dramas to romantic comedies - all in the Freeform app!\n \nFULL EPISODES: Enjoy the best lineup of dramas, hilarious comedies and binge throwback series.  Stream the latest episodes* of your favorite Freeform TV shows like:\n \nMotherland\nSiren\nThe Bold Type\nAlone To

In [263]:
driver.close()
driver.quit()

In [ ]:
# all_apps_list[0]
insertToApp([
{
        "appstore_link": "https://channelstore.roku.com/details/b1849971f841d4c61f35a14fbf6f8b91",
        "developer_name": "Oli Tormey",
        "developer_page": None,
        "ottRevenueModel": "",
        "ottBroadcastType": "LIVE",
        "ottProductionType": "UGC",
        "dspAppId": "184833E",
        "appstoreAppName": "Church Services TV",
        "appstoreAppId": "b1849971f841d4c61f35a14fbf6f8b91",
        "insertDate": "2023-03-06T08:17:43.581635Z",
        "updateDate": "2023-03-06T08:17:43.581635Z",
        "type": "Manual",
        "appAverageRating": 4.075,
        "appNumRaters": 40,
        "appDescription": "Church Services TV have been streaming from Churches since 2007. Every day and every hour you can watch LIVE, Churches from all over the UK, Ireland, and the USA. ",
        "ctvStoreName": "Roku",
        "ctvStoreId": 3,
        "ottDevicePlatformCtv": "Roku",
        "ottContentCategories": "Religion;Movies;TV SHOWS;"
    }
])

In [ ]:
all_apps_list[0]

In [ ]:
## DRAFTS##

In [ ]:
URL_LIST = ["https://channelstore.roku.com/en-ca/details/0242a278cfe3c00df08bd29bafb25b79/cbc-tv", "https://channelstore.roku.com/en-ca/details/96da35e0bce6c184b61e445cc6e62203/netflix"]

In [ ]:
driver = webdriver.Chrome('\\Users\Eran.Braude\Downloads\chromedriver_win32/chromedriver.exe')

In [ ]:
roku_apps_list = []
for url in URL_LIST:
    roku_app = {}
    try:
        driver.get(url)
        time.sleep(5)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        soup = BeautifulSoup(html, "lxml")
        #Appstore app name
        name = soup.find('h1', attrs = {"itemprop" : "name"}).text
        print(name)
        #rating
        rating = soup.find('span', attrs = {"itemprop" : "averageRating", "class" : "average-rating"}).text
        print(rating)
        #Raters
        raters = soup.find('div', attrs = {"itemtype" : "https://schema.org/Review"}).contents[1].text[3:-len(' ratings')]
        print(raters)
    except Exception as e:
        print(f"{e}")

In [ ]:
driver.get("https://channelstore.roku.com/en-ca/details/136db0bee0360139f81fba8e7e95786d/cbc-tv")
time.sleep(5)
content = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(content, "lxml")
categories = soup.find('p', attrs = {"class" : "categories"}).find_all('a')
categories_list = []
for category in categories:
    categories_list.append(category.text)
print(categories_list)

In [ ]:
x = soup.find('div', attrs = {"itemtype" : "https://schema.org/Review"}).contents[1].text[3:-len(' ratings')]
print(x)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
opts = Options()
opts.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36')
driver = webdriver.Chrome(chrome_options=opts)

In [ ]:
driver.get("https://channelstore.roku.com/en-ca/details/96da35e0bce6c184b61e445cc6e62203/netflix")
# time.sleep(10)
# driver.implicitly_wait(220)
content = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
# html = driver.page_source
soup = BeautifulSoup(content, "lxml")
soup

In [ ]:
URL = "https://us.lgappstv.com/main/tvapp/detail?appId=1175168"

In [ ]:
driver = webdriver.Chrome('\\Users\Eran.Braude\Downloads\chromedriver_win32/chromedriver.exe')

In [ ]:
driver.get(URL)
sec_html = driver.page_source
sec_soup = BeautifulSoup(sec_html, "lxml")

In [ ]:
driver.quit()

In [ ]:
myDict = [{'Title': None, 'Score': 'A-'}, {'Title': 'The shine', 'Score': 'B'}, {'Title': 'Test Movie', 'Score': 'C-'}]
df = pd.DataFrame.from_records(myDict)

In [ ]:
df

In [ ]:
test = {}

In [ ]:
test['name'] = 'Harry Potter'

In [ ]:
test['score'] = 'A-'

In [ ]:
for x in test.values():
    print(x)

In [ ]:
{
    "appstore_app_id": "100532",
    "appstore_app_name": "Mystery Movie Theatre",
    "app_long_id": "37c826fa9f30eb54b92c4f2f2ea65c34",
    "app_average_rating": 3.5,
    "app_num_raters": 227,
    "app_description": "Classic mysteries are the perfect snuggle up Friday night movie. Every other night we just shake in fear. The original classic mystery movies are the best. Mystery Movie Theatre channel is always free for everyone to enjoy thanks to our advertising sponsors. For more information please visit, Privacy Policy: http://phoenixentertainment.org/privacy Opt-Out Policy: http://phoenixentertainment.org/optout ",
    "ctv_store_name": "Roku",
    "ctv_store_id": 3,
    "ott_device_platform_ctv": "Roku",
    "ott_production_type": "Pro-Am",
    "ott_content_categories": "Movies & TV;",
    "appstore_link": "https://channelstore.roku.com/details/37c826fa9f30eb54b92c4f2f2ea65c34",
    "developer_name": "The Movie Club",
    "developer_page": "docs.roku.com",
    "ctv_ott": "OTT-CTV",
    "non_ott_sub_category": null
}

In [ ]:
def testMe(x):
    '''
    desc:
    param:
    return:
    '''
    xTest = 5 + x
    print(xTest)
    


In [ ]:
from selenium import webdriver
chrome_options = Options()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(chrome_options=chrome_options)

In [ ]:
response = req.get('https://channelstore.roku.com/api/v6/channels/0242a278cfe3c00df08bd29bafb25b79', headers = headers, proxies = proxies)
test = json.loads(response.content)
print (json.dumps(test, indent=4))

In [ ]:
driver.get('https://channelstore.roku.com/en-gb/details/136db0bee0360139f81fba8e7e95786d/wwe-network')
time.sleep(1)
driver.get('https://channelstore.roku.com/api/v6/channels/136db0bee0360139f81fba8e7e95786d')
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
app_json = soup.find('pre').text
test = json.loads(app_json)
print (json.dumps(test, indent=4))

In [ ]:
driver.quit();

In [ ]:
"{:.1f}".format(42.25478654)

In [ ]:
def fun_that_return_two_values():
    print('Hi!')
    return 100, True

In [ ]:
val1, val2 = fun_that_return_two_values()
type(val2)

In [ ]:
# apple_apps['appstore_app_name_clean'] = [i[0:i.index(":")] if ":" in i else i for i in apple_apps['appstore_app_name'].tolist()]
test = 'eran : braude'
name = test[0:test.index(":")]
name

In [ ]:
testDict = {'game' : 'Games', 'play' : 'Games'}

Teststr = 'this is a vool video gamee.'

for val in testDict:
    if f' { val.lower() } ' in Teststr.lower():
        print(testDict[val])

In [ ]:
testDict = {'game' : 'Games', 'play' : 'Games'}

test = testDict.get('game', 0)

test

In [ ]:
def send_API_build_app_obj(param_id):
    '''
    desc: The function will create an URL and send it to the Roku API. Process the response and make the initial app object.
    param: long id.
    return: initial app object.
    '''
    categories_list = []
    
    url_to_api = f'https://channelstore.roku.com/api/v6/channels/{param_id}'
    driver.get(f'https://channelstore.roku.com/en-ca/details/{param_id}/cbc-tv')
    time.sleep(1)
    driver.get(url_to_api)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    app_json = soup.find('pre').text
    response = json.loads(app_json)
    
#     response = json.loads(sess.get(url_to_api, headers = headers, proxies = proxies).content)
#     print (json.dumps(response, indent=4))

    categories = response['categoryList']
    if(response['categoryList'] == None):
        categories_list.append(None)
    else:
        for category in response['categoryList']:
            categories_list.append(category['name'])
            
    app_object = {'appstore_app_id' : response['id'],
                 'appstore_app_name' : response['name'],
                 'app_long_id' : param_id,
                 'app_average_rating' : (response['rating'] / 10) / 2, #"{:.1f}".format(42.25478654)
                 'app_num_raters' : response['starRatingCount'],
                 'app_description' : response['webDescription'],
                 'ctv_store_name' : 'Roku',
                 'ctv_store_id' : 3,
                 'ott_device_platform_ctv' : 'Roku',
                'ott_content_categories' : categories_list,
                 'appstore_link' : f'https://channelstore.roku.com/details/{param_id}',
                 'developer_name' : response['developerName'],
                 'developer_page' : response['currentDetail']['developerPrivacyUrl'],
                 }
    
    print (json.dumps(app_object, indent=4))
    return app_object

In [ ]:
f = open('LALALALALA.json')
json.load(f)

In [ ]:
config = open('Utils_Roku_For_Crawler.json')
config_file = json.load(config)
config_file
with open("Utils_Roku_For_Crawler.json", "w") as write_file:
    json.dump(config_file, write_file, indent=4)

In [ ]:
from datetime import datetime, timezone

x = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

# d = datetime.today() - timedelta(days=1)
print(x)

In [ ]:
def bring_app_properties_categories_dana(app_obj):
    
    categories_word_to_category = config_file['categories_word_to_category']
    
#     {'news' : 'news', 'movies' : 'Movies', 'TV shows' : 'TV Shows', 'movie' : 'Movies', 'TV show' : 'TV Shows',
#                                   'documentaries' : 'Documentary', 'Documentary' : 'Documentary', 'kids' : 'Kids', 'Entertainment' : 'Entertainment',
#                                   'anime' : 'Anime', 'shows' : 'TV Shows', 'family' : 'Family', 'game shows' : 'Game Shows', 'news' : 'news', 'sports' : 'Sports',
#                                   'music' : 'Music', 'educational' : 'Education', 'cartoons' : 'Cartoon', 'cinema' : 'Movies',
#                                   'Film' : 'Movies', 'all genres' : 'Entertainment', 'WWE' : 'Boxing', 'WCW' : 'Boxing', 'ECW' : 'Boxing',
#                                   'ring shows' : 'Sports', 'food' : 'Food&Beverage', 'food shows' : 'Food&Beverage', 'cooking' : 'Food&Beverage', 'recipes' : 'Food&Beverage', 'cuisines' : 'Food&Beverage', 'vegan' : 'Food&Beverage', 'foodies' : 'Food&Beverage',
#                                   'cartoon' : 'Cartoon', 'parent friendly' : 'Kids', 'Toon' : 'Cartoon', 'kidSAFE' : 'Kids', 'Headlines' : 'news',
#                                   'Journalist' : 'news', 'breaking news' : 'news', 'weather' : 'Weather', 'celebrities' : 'Celebrities and Gossip', 'reality programming' : 'Reality', 'home improvement' : 'Home&Garden', 'Horror' : 'Horror', 'Sci Fi' : 'Sci-Fi',
#                                   'films' : 'Movies', 'slashers' : 'Horror', 'docuseries' : 'Documentary', "Christian" : 'Religion', 'Faith' : 'Religion', 'Fitness' : 'Fitness', 'workouts' : 'Fitness', 'Church' : 'Religion', 'NFL' : 'AmericanFootball', 'comedies' : 'Comedy', 'Comedy' : 'Comedy',
#                                   'reality TV' : 'Reality', 'reality shows' : 'Reality', 'Fishing' : 'Recreation', 'series' : 'TV Shows', 'Gospel' : 'Religion',
#                                   'Catholic' : 'Religion', 'Entertaining' : 'Entertainment'}

    app_obj['app_description_clean'] = clean_text_from_special_chars(app_obj['desc'])
    categories_list = []
    
    for word in categories_word_to_category:
        if f' {word.lower()} ' in app_obj['app_description_clean'].lower():
            categories_list.append(categories_word_to_category[word])
    
    if(len(categories_list) > 0):
        categories_list = list(dict.fromkeys(categories_list))
        app_obj["ott_content_categories"] = categories_list
        
    else:
        categories_list_sec = []
        categories_list = []
        for roku_category in app_obj["ott_content_categories"]:
            for classifier_category in config_file['content_categories_to_category']:
                if(roku_category == classifier_category):
                    categories_list.append(config_file['content_categories_to_category'][classifier_category])
                    
        for cat_list in categories_list:
            for categoey in cat_list:
                categories_list_sec.append(categoey)    
                
        app_obj["ott_content_categories"] = categories_list_sec

    return app_obj

In [ ]:
matrix = [[j for j in range(5)] for i in range(5)]

In [ ]:
df_dana = pd.read_excel('C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Dana_Tag.xlsx')

In [ ]:
df_dana = df_dana[['dsp_app_id', 'desc']]

In [ ]:
df_dana
df_dana_2 = df_dana.to_json(orient = 'records', indent=4)

In [ ]:
dana_dict = json.loads(df_dana_2)

In [ ]:
desc = 'Here comes another delightfully intriguing gem of Mascot Pictures, The Adventures of Rex and Rinty. This is a 12-episode sound serial starring two wonderful animals, a horse, and a dog. Rex, the King of the Wild Horses, is forced to the U.S. from a faraway island of Sujan, where he used to be worshipped as a divine creature. When people wanted to train him as a polo pony, the stallion refused to obey and escaped. Soon he met Rin-Tin-Tin, Jr., also known as Rinty, the vagabond dog. The sacred Arabian horse dreams about returning back to the wrap-around safety of the jungle. Thanks to the intervention of polo-player Frank Bradley, both animals are safely taken back to the stallion’s island. However, that’s only the beginning of their adventure… Are you curious about the rest of the story? Luckily, there’s a chance to lift the veil and take a peek behind the curtain. Our free online channel – Adventurers Rex and Rinty – is a shortcut to the black-and-white world of freedom, loyalty, and friendship. Subscribe to this channel for more episodes of your favorite sound serial and spend quality time online.  '

bring_app_properties_categories_dana({'dsp_app_id': '612432',
 'desc': desc,
  "ott_content_categories": [
        "Film & TV"
    ]})

In [ ]:
for app in dana_dict:
    bring_app_properties_categories_dana(app)

In [ ]:
dana_dict

In [ ]:
df = pd.DataFrame.from_records(dana_dict)
df.to_excel('C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Dana_Tag_result.xlsx', index=False)

In [ ]:
Teststr = " be a part of it watch along the premiere of mifs and crashland pictures original film uncanny harbor \n\nset in a seaside new england town a fisherman suffers from visions of his wife who has been missing for 25 years with her disappearance remaining a mystery investigators work off the few clues left behind playing with themes of isolation and paranoia uncanny harbor explores a world shrouded with unanswered questions while walking the line between science fiction and horror \n\nwatch our trailers awaiting our world premiere this february \n\nrent uncanny harbor on 02 22 20 and share in this unique experience \n\nterms of service https www uncannyharbor watchtos\nprivacy policy https www uncannyharbor watchprivacy\n "

In [ ]:
Teststr

In [ ]:
Teststr

" \n\nrent " in Teststr

In [80]:
def bring_long_id(param_id):
    '''
    desc: The function will return hashed Roku store id, based on a data frame that mapping between short id and long id.
    param: id as it comes from ID_LIST.
    return: long id.
    '''
    df = pd.read_csv("C:\\Users\\Eran.Braude\\Dropbox\\Weka\\Roku_ID\\Roku_All_id.csv", squeeze = True)
    df['Roku_id'] = df['Roku_id'].astype(int)
    return df.head(5)
#     df['Roku_id'] = df['Roku_id'].astype(str)
#     if(len(param_id) > 10):
#         return param_id
#     else:
#         try:
#              print(df[df['Roku_id'] == param_id])
# #             return df[df['Roku_id'] == param_id]['long_id'].tolist()[0]
#         except Exception as e:
#             print(f"{e}")
#             return -1 

In [81]:
bring_long_id('63402')

ValueError: Cannot convert non-finite values (NA or inf) to integer